In [2]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.0.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.0.0-cp312-cp312-win_amd64.whl (16.2 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [4]:
!pip install scikit-learn

  Using cached scikit_learn-1.5.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.0-cp312-cp312-win_amd64.whl (10.9 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [4]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('./data/train_tfidf_features.csv')

df_feature = df.drop('label', axis=1)
df_feature = df_feature.drop('id', axis=1)
df_target = pd.DataFrame(df['label'])


In [57]:
df_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df_target.head()

,label
0,1
1,0
2,1
3,0
4,1


In [59]:
def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    indexes = df_feature.index.to_numpy()
    if random_state is not None:
        np.random.seed(random_state)
    test_index = np.random.choice(indexes, int(len(indexes) * test_size), replace=False)
    train_index = np.setdiff1d(indexes, test_index)
    
    df_feature_train = df_feature.loc[train_index, :]
    df_feature_test = df_feature.loc[test_index, :]
    df_target_train = df_target.loc[train_index, :]
    df_target_test = df_target.loc[test_index, :]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

def normalize_z(dfin, columns_means=None, columns_stds=None):
    if columns_means is None:
        columns_means = dfin.mean(axis=0)
    if columns_stds is None:
        columns_stds = dfin.std(axis=0)
    
    # Prevent division by zero
    columns_stds = columns_stds.replace(0, 1e-5)
    
    dfout = (dfin - columns_means) / columns_stds
    return dfout, columns_means, columns_stds

# Task 3 L

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Load the dataset
df = pd.read_csv('./data/train_tfidf_features.csv')

# Separate features and labels
X = df.drop('label', axis=1)  # Replace 'label_column' with the actual label column name
y = df['label']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to apply PCA
def apply_pca(n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# Apply PCA for different component sizes
components = [2000, 1000, 500, 100]
pca_results = {n: apply_pca(n) for n in components}

# Train KNN and evaluate
knn = KNeighborsClassifier(n_neighbors=2)

def train_and_evaluate(X_train_pca, X_test_pca):
    knn.fit(X_train_pca, y_train)
    y_pred = knn.predict(X_test_pca)
    return y_pred

results = {}
for n, (X_train_pca, X_test_pca) in pca_results.items():
    y_pred = train_and_evaluate(X_train_pca, X_test_pca)
    f1 = f1_score(y_test, y_pred, average='macro')
    results[n] = f1

# Print results
for n, f1 in results.items():
    print(f"PCA Components: {n}, Macro F1 Score: {f1}")

# Save predictions for Kaggle submission (example for 100 components)
y_pred_100 = train_and_evaluate(*pca_results[100])
submission = pd.DataFrame({'Id': X_test.index, 'Prediction': y_pred_100})
submission.to_csv('submission_100_components.csv', index=False)


PCA Components: 2000, Macro F1 Score: 0.47577897934068236
PCA Components: 1000, Macro F1 Score: 0.47165413769915304
PCA Components: 500, Macro F1 Score: 0.473529103715535
PCA Components: 100, Macro F1 Score: 0.476917766582954


In [10]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.7/124.9 MB 20.8 MB/s eta 0:00:06
    --------------------------------------- 1.8/124.9 MB 23.2 MB/s eta 0:00:06
    --------------------------------------- 3.1/124.9 MB 24.6 MB/s eta 0:00:05
   - -------------------------------------- 4.2/124.9 MB 24.6 MB/s eta 0:00:05
   - -------------------------------------- 5.6/124.9 MB 25.5 MB/s eta 0:00:05
   -- ------------------------------------- 6.7/124.9 MB 25.4 MB/s eta 0:00:05
   -- ------------------------------------- 8.0/124.9 MB 25.5 MB/s eta 0:00:05
   -- ------------------------------------- 9.1/124.9 MB 25.4 MB/s eta 0:00:05
   --- ------------------------------------ 9.6/124.9 MB 25.6 MB/s eta 0:00:05
   --- ------------------------------------ 9.7/124.9 MB 22.3 MB/s eta 0:00:06
   --- ------------------------------------ 9.9/124.9 MB 20.4 MB/s eta 0:00:06
   --- ------------------------------------ 10.2/124.9 MB 18

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

# Load the dataset
df = pd.read_csv('./data/train_tfidf_features.csv')

# Separate features and labels
X = df.drop('label', axis=1)  # Replace 'label' with the actual label column name
y = df['label']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Function to apply PCA
def apply_pca(n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# Apply PCA for different component sizes
components = [2000, 1000, 500, 100]
pca_results = {n: apply_pca(n) for n in components}

# Grid search for KNN hyperparameters
param_grid = {'n_neighbors': [1, 2, 3, 5, 7, 10]}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, scoring='f1_macro', cv=5)

# Train KNN and evaluate
results = {}
for n, (X_train_pca, X_test_pca) in pca_results.items():
    grid_search.fit(X_train_pca, y_train)
    best_knn = grid_search.best_estimator_
    y_pred = best_knn.predict(X_test_pca)
    f1 = f1_score(y_test, y_pred, average='macro')
    results[n] = f1

# Print KNN results
for n, f1 in results.items():
    print(f"KNN with PCA Components: {n}, Macro F1 Score: {f1}")

# Evaluate with Random Forest, SVM, and XGBoost
def evaluate_model(model, X_train_pca, X_test_pca):
    model.fit(X_train_pca, y_train) 
    y_pred = model.predict(X_test_pca)
    return f1_score(y_test, y_pred, average='macro')

rf_results = {n: evaluate_model(RandomForestClassifier(), X_train_pca, X_test_pca) for n, (X_train_pca, X_test_pca) in pca_results.items()}
svm_results = {n: evaluate_model(SVC(), X_train_pca, X_test_pca) for n, (X_train_pca, X_test_pca) in pca_results.items()}
xgb_results = {n: evaluate_model(XGBClassifier(), X_train_pca, X_test_pca) for n, (X_train_pca, X_test_pca) in pca_results.items()}

# Print results for other models
print("Random Forest Results:")
for n, f1 in rf_results.items():
    print(f"PCA Components: {n}, Macro F1 Score: {f1}")

print("SVM Results:")
for n, f1 in svm_results.items():
    print(f"PCA Components: {n}, Macro F1 Score: {f1}")

print("XGBoost Results:")
for n, f1 in xgb_results.items():
    print(f"PCA Components: {n}, Macro F1 Score: {f1}")

# Save best predictions for Kaggle submission (example for 100 components)
y_pred_100 = best_knn.predict(pca_results[100][1])
submission = pd.DataFrame({'Id': X_test.index, 'Prediction': y_pred_100})
submission.to_csv('submission_100_components.csv', index=False)


KNN with PCA Components: 2000, Macro F1 Score: 0.5218488011865823
KNN with PCA Components: 1000, Macro F1 Score: 0.5176021928458235
KNN with PCA Components: 500, Macro F1 Score: 0.5527413341275035
KNN with PCA Components: 100, Macro F1 Score: 0.593241729056758
Random Forest Results:
PCA Components: 2000, Macro F1 Score: 0.5415740149442299
PCA Components: 1000, Macro F1 Score: 0.5640225365950495
PCA Components: 500, Macro F1 Score: 0.5892665313150522
PCA Components: 100, Macro F1 Score: 0.6091303306873392
SVM Results:
PCA Components: 2000, Macro F1 Score: 0.6402529331404694
PCA Components: 1000, Macro F1 Score: 0.6324128771807379
PCA Components: 500, Macro F1 Score: 0.6302789501203805
PCA Components: 100, Macro F1 Score: 0.6183301523100327
XGBoost Results:
PCA Components: 2000, Macro F1 Score: 0.6217998402477325
PCA Components: 1000, Macro F1 Score: 0.6394975168822356
PCA Components: 500, Macro F1 Score: 0.62840512362262
PCA Components: 100, Macro F1 Score: 0.6311885635259854


AttributeError: 'numpy.ndarray' object has no attribute 'index'

# Logistic Regression Task 1

In [24]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('./data/train_tfidf_features.csv')

df_feature = df.drop('label', axis=1)
df_feature = df_feature.drop('id', axis=1)
target = pd.DataFrame(df['label'])


In [25]:
df_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
target.head()

,label
0,1
1,0
2,1
3,0
4,1


In [27]:
import numpy as np
import pandas as pd

def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    indexes = df_feature.index.to_numpy()
    if random_state is not None:
        np.random.seed(random_state)
    test_index = np.random.choice(indexes, int(len(indexes) * test_size), replace=False)
    train_index = np.setdiff1d(indexes, test_index)
    
    df_feature_train = df_feature.loc[train_index, :]
    df_feature_test = df_feature.loc[test_index, :]
    df_target_train = df_target.loc[train_index, :]
    df_target_test = df_target.loc[test_index, :]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

def normalize_z(dfin, columns_means=None, columns_stds=None):
    if columns_means is None:
        columns_means = dfin.mean(axis=0)
    if columns_stds is None:
        columns_stds = dfin.std(axis=0)
    
    # Prevent division by zero
    columns_stds = columns_stds.replace(0, 1e-5)
    
    dfout = (dfin - columns_means) / columns_stds
    return dfout, columns_means, columns_stds

def prepare_feature(df_feature):
    if isinstance(df_feature, pd.DataFrame):
        np_feature = df_feature.to_numpy()
    else:
        np_feature = df_feature
    X = np.hstack((np.ones((np_feature.shape[0], 1)), np_feature))
    return X

def prepare_target(df_target):
    if isinstance(df_target, pd.DataFrame):
        np_target = df_target.to_numpy()
    else:
        np_target = df_target
    return np_target.ravel()

def calc_logreg(X, beta):
    z = np.dot(X, beta)
    p_x = 1 / (1 + np.exp(-z))
    return p_x

def compute_cost_linreg(beta, X, y):
    epsilon = 1e-5
    pred = np.clip(calc_logreg(X, beta), epsilon, 1 - epsilon)
    error = np.where(y == 1, np.log(pred), np.log(1 - pred))
    J = -np.mean(error)
    return J

def gradient_descent_logreg(X, y, beta, alpha, num_iters):
    m = y.shape[0]
    J_storage = np.zeros(num_iters)
    for i in range(num_iters):
        pred = calc_logreg(X, beta)
        error = pred - y
        gradient = np.dot(X.T, error) / m
        beta -= alpha * gradient
        J_storage[i] = compute_cost_linreg(beta, X, y)
        
        # Debugging: print intermediate values
        if i % 100 == 0:
            print(f"Iteration {i}: Cost {J_storage[i]}, Beta {beta[:5]}")  # Print the first 5 beta values for readability
        
    return beta, J_storage

def predict_norm(X, beta):
    probabilities = calc_logreg(X, beta)
    return np.where(probabilities >= 0.5, 1, 0)

def predict_logreg(df_feature, beta, means=None, stds=None):
    df_feature, means, stds = normalize_z(df_feature, means, stds)
    X = prepare_feature(df_feature)
    return predict_norm(X, beta)

# Sample usage:
# Assuming df_feature and target are defined
df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(df_feature, target, random_state=42, test_size=0.3)

df_feature_train_norm, means, stds = normalize_z(df_feature_train)

X = prepare_feature(df_feature_train_norm)
y = prepare_target(df_target_train)

beta = np.zeros(X.shape[1])
alpha = 0.01
num_iters = 3000
beta, J_storage = gradient_descent_logreg(X, y, beta, alpha, num_iters)

predictions = predict_logreg(df_feature_test, beta, means, stds)

# Print final beta and cost
print(f"Final Beta: {beta}")
print(f"Final Cost: {J_storage[-1]}")



Iteration 0: Cost 0.6908383573948907, Beta [-1.15761909e-03  1.92035294e-05 -8.05249720e-05  2.22213018e-05
  2.93724623e-05]
Iteration 100: Cost 0.5424007585248339, Beta [-0.10365702  0.00078116 -0.00655588  0.00174863  0.00219551]
Iteration 200: Cost 0.47294474115931523, Beta [-0.18506657  0.00018589 -0.01094772  0.0027942   0.00335115]
Iteration 300: Cost 0.4321545291054839, Beta [-0.25140195 -0.000961   -0.01418896  0.00346084  0.00396459]
Iteration 400: Cost 0.4048276883917934, Beta [-0.30666288 -0.00229727 -0.01674394  0.00390978  0.00426686]
Iteration 500: Cost 0.3849768466777291, Beta [-0.35354844 -0.00365603 -0.01885948  0.00422482  0.00438225]
Iteration 600: Cost 0.36975335996353237, Beta [-0.39394475 -0.00496153 -0.02067617  0.00445092  0.00438329]
Iteration 700: Cost 0.3576167651733698, Beta [-0.4292121  -0.0061826  -0.02227924  0.00461355  0.00431482]
Iteration 800: Cost 0.34765503941402215, Beta [-0.46035752 -0.00731003 -0.02372327  0.00472821  0.00420553]
Iteration 900: 

In [28]:
beta

array([-0.77581454, -0.01892811, -0.04304434, ..., -0.02167527,
       -0.03465155,  0.03996443])

In [29]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(df_target_test, predictions)
print(cm)

[[2406  820]
 [ 814 1115]]


In [30]:
from sklearn.metrics import f1_score
f1_score(df_target_test, predictions, average='macro')

np.float64(0.6618158082210612)

In [31]:
df_test_set = pd.read_csv('./data/test_tfidf_features.csv')
df_test_set = df_test_set.drop('id', axis=1)
predictions = predict_logreg(df_test_set, beta, means, stds)
predictions

array([0, 0, 1, ..., 1, 0, 0])

In [33]:
predictions_df = pd.DataFrame(predictions, columns=['label'])
test_id = pd.read_csv('./data/test_tfidf_features.csv')['id']
submission_df = pd.concat([test_id, predictions_df], axis=1)
submission_df.to_csv('LogRed_Prediction.csv', index=False)

---

# Task 2

In [9]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

## PCA and KNearestNeighbours (KNN)

PCA:

PCA in sklearn takes in these arguments:

1. n_components: int/float
    - Number of components to keep. By deafult is all components
    
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

In [11]:
df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(df_feature, df_target, random_state=42, test_size=0.3)

scaler = StandardScaler()
df_feature_train_scaled = scaler.fit_transform(df_feature_train)
df_feature_test_scaled = scaler.transform(df_feature_test)

KNN:

Most imporatantly, KNeighborsClassifier in sklearn takes in these arguments: 
1. n_neighours: int
    - the number of neighbours that we will be comparing to for us to determine how to classify the identified point
2. weights: ['uniform', 'distance']
    - uniform: All points in each neighborhood are weighted equally
    - distance: Weigh points by the inverse of their distance. --> Closer neighbors of a query point will have greater influence than neighbors which are further away
3. metrics: str
    - metric to use for distance computation. Default is minkowski

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier


In [39]:
n_components = [2000, 1000, 500, 100]
f1_dict = {'uniform, minkowski': [0, 0, 0, 0],
           'uniform, euclidean': [0, 0, 0, 0],
           'uniform, manhattan': [0, 0, 0, 0],
           'distance, minkowski': [0, 0, 0, 0],
           'distance, euclidean': [0, 0, 0, 0],
           'distance, manhattan': [0, 0, 0, 0]}

for i, n in enumerate(n_components):
    pca = PCA(n_components=n)
    df_feature_train_pca = pca.fit_transform(df_feature_train_scaled)
    df_feature_test_pca = pca.transform(df_feature_test_scaled)
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='uniform', metric='minkowski')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: uniform, metrics: minkowski, Macro F1 Score: {macro_f1}")
    f1_dict['uniform, minkowski'][i] = macro_f1
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='uniform', metric='euclidean')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: uniform, metrics: euclidean, Macro F1 Score: {macro_f1}")
    f1_dict['uniform, euclidean'][i] = macro_f1
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='uniform', metric='manhattan')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: uniform, metrics: manhattan, Macro F1 Score: {macro_f1}")
    f1_dict['uniform, manhattan'][i] = macro_f1
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='distance', metric='minkowski')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: distance, metrics: minkowski, Macro F1 Score: {macro_f1}")
    f1_dict['distance, minkowski'][i] = macro_f1
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='distance', metric='euclidean')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: distance, metrics: euclidean, Macro F1 Score: {macro_f1}")
    f1_dict['distance, euclidean'][i] = macro_f1
    
    knn = KNeighborsClassifier(n_neighbors=2, weights='distance', metric='manhattan')
    knn.fit(df_feature_train_pca, df_target_train)
    y_pred = knn.predict(df_feature_test_pca)
    
    macro_f1 = f1_score(df_target_test, y_pred, average='macro')
    print(f"PCA Components: {n}, weight: distance, metrics: manhattan, Macro F1 Score: {macro_f1}")
    f1_dict['distance, manhattan'][i] = macro_f1
    # submission = pd.DataFrame({'Id': np.arange(len(y_pred)), 'Predicted': y_pred})
    # submission.to_csv(f'./knn_submissions/knn_pca_{n}_components_submission.csv', index=False)
    
    
    


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


PCA Components: 2000, weight: uniform, metrics: minkowski, Macro F1 Score: 0.5501698573624343


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


PCA Components: 2000, weight: uniform, metrics: euclidean, Macro F1 Score: 0.5501698573624343


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


...

PCA Components: 100, weight: uniform, metrics: manhattan, Macro F1 Score: 0.5452027672093189
PCA Components: 100, weight: distance, metrics: minkowski, Macro F1 Score: 0.5817419006772417


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


PCA Components: 100, weight: distance, metrics: euclidean, Macro F1 Score: 0.5817419006772417


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


PCA Components: 100, weight: distance, metrics: manhattan, Macro F1 Score: 0.5765390074323072


In [40]:
f1_dict

{'uniform, minkowski': [np.float64(0.5501698573624343),
  np.float64(0.5509358763631034),
  np.float64(0.5537058356694703),
  np.float64(0.5486275090215184)],
 'uniform, euclidean': [np.float64(0.5501698573624343),
  np.float64(0.5509358763631034),
  np.float64(0.5537058356694703),
  np.float64(0.5486275090215184)],
 'uniform, manhattan': [np.float64(0.5524122193964222),
  np.float64(0.5508372124746364),
  np.float64(0.5517738757177308),
  np.float64(0.5452027672093189)],
 'distance, minkowski': [np.float64(0.5392491953449301),
  np.float64(0.5501770344051041),
  np.float64(0.5605568943455942),
  np.float64(0.5817419006772417)],
 'distance, euclidean': [np.float64(0.5392491953449301),
  np.float64(0.5501770344051041),
  np.float64(0.5605568943455942),
  np.float64(0.5817419006772417)],
 'distance, manhattan': [np.float64(0.5435481359602898),
  np.float64(0.5495024023827315),
  np.float64(0.560207813194894),
  np.float64(0.5765390074323072)]}

In [23]:
from sklearn.model_selection import GridSearchCV, cross_val_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_feature_train)
X_test_scaled = scaler.transform(df_feature_test)

# Function to perform PCA and KNN, and return the Macro F1 score
def pca_knn(X_train, y_train, X_test, y_test, n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=2)
    knn.fit(X_train_pca, y_train)
    y_pred = knn.predict(X_test_pca)
    
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    return macro_f1, y_pred

# Function to perform hyperparameter tuning and cross-validation
def tune_knn(X_train, y_train, X_test, y_test, n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Define parameter grid
    param_grid = {
        'n_neighbors': [2],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'minkowski' , 'manhattan']
    }
    
    knn = KNeighborsClassifier()
    grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')
    grid_search.fit(X_train_pca, y_train)
    
    best_knn = grid_search.best_estimator_
    y_pred = best_knn.predict(X_test_pca)
    
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    return macro_f1, y_pred, grid_search.best_params_

# Number of components to try
components_list = [2000, 1000, 500, 100]
macro_f1_scores = {}
best_params = {}
predictions = {}

for n_components in components_list:
    macro_f1, y_pred, best_params_for_config = tune_knn(X_train_scaled, df_target_train, X_test_scaled, df_target_test, n_components)
    macro_f1_scores[n_components] = macro_f1
    best_params[n_components] = best_params_for_config
    predictions[n_components] = y_pred

# Print the Macro F1 scores and best parameters
for n_components in components_list:
    print(f"Macro F1 score with {n_components} components: {macro_f1_scores[n_components]}")
    print(f"Best parameters with {n_components} components: {best_params[n_components]}")

# Save predictions for Kaggle submission
for n_components, y_pred in predictions.items():
    submission = pd.DataFrame({'Id': np.arange(len(y_pred)), 'Predicted': y_pred})
    submission.to_csv(f'./knn_true/knn_pca_{n_components}_components_submission.csv', index=False)

c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example us

Macro F1 score with 2000 components: 0.5427743603294302
Best parameters with 2000 components: {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'distance'}
Macro F1 score with 1000 components: 0.5450593232052263
Best parameters with 1000 components: {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'distance'}
Macro F1 score with 500 components: 0.5499751045702174
Best parameters with 500 components: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'}
Macro F1 score with 100 components: 0.5747690063350932
Best parameters with 100 components: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'}


In [21]:
macro_f1_scores
best_params
# predictions

{2000: {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'distance'},
 1000: {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'distance'},
 500: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'},
 100: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'}}

# Task 3

## GridSearchCV

sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring=None, n_jobs=None, refit=True, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

1. estimator: estimator object
    - Machine learning model from sklearn that you set to a variable
2. param_grid: dictionary or list of dictionaries
    - dictionary with parameter names as keys and lists of parameter settings to try as values
    - a list of such dictionaries in which case the grids spanned by each dictionary in the list are explored
3. scoring: str
    - single string to represent the scorer
    - a list or tuple of unique strings for multiple scorers
4. n_jobs: int
    - default = None
    - number of jobs to run in parallel
5. cv: int
    - default = None
    - determines the cross-validation splitting strategy
    - integer represents number of folds

Attributes:

1. gridsearch.best_estimator_
    - estimator that gives the highest score
    - it is the model with the params that give the best results
2. gridsearch.best_score_
    - mean cross-validated 
3. gridsearch.best_params_
    - returns a dictionary of the best params that make up the best estimator

Functions:
1. .fit(X, y=None, **params)
    - X: array-like of shape (n_samples, n_features)
    - y: array-like of shape (n_samples, n_features) or (n_samples, ), default = None
    - **params: dictonary of string
        * idk what it does so not important??
2. .predict(X)
    - X: indexable, length of n_samples
    - returns y_pred: ndarray of shape (n_samples)

## Standard Vector Machine (SVM)

SVC:

sklearn.svm.SVC():

1. C: float
    - regularization parameter
    - default = 1.0
    - strength of the regularization is inversely proportional to C
2. kernal
    - kernal to be used
    - default is 'rbf'
3. degree: int
    - default = 3
    - degree of the polynomal kernal function
4. random_state: int
    - deafult = None
    - controls the pseudo random number generator for shuffling data

In [46]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(df_feature, target, random_state=42, test_size=0.3)
scalar = StandardScaler()
df_feature_train_scaled = scalar.fit_transform(df_feature_train)
df_feature_test_scaled = scalar.transform(df_feature_test)

# Initialize the SVM model
svm = SVC(kernel='linear', C=1, random_state=42)

In [49]:
# Train the model
svm.fit(df_feature_train_scaled, df_target_train)

# Make predictions
y_pred = svm.predict(df_feature_test_scaled)

c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

## XGBoost

In [1]:
!pip install xgboost


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import xgboost as XGBClassifier
from sklearn.preprocessing import StandardScaler

df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(df_feature, df_target, random_state=42, test_size=0.3)
scalar = StandardScaler()
df_feature_train_scaled = scalar.fit_transform(df_feature_train)
df_feature_test_scaled = scalar.transform(df_feature_test)

In [ ]:
model = XGBClassifier(objective='binary:logistic', learning_rate=0.1, n_estimators=100, max_depth=5)

In [ ]:
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [16]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Create a pipeline with PCA and XGBoost
pipeline = Pipeline([
    ('pca', PCA()), 
    ('classifier', xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.1))
])

# Create a parameter grid
param_grid = {
    'pca__n_components': [2000, 1000, 500, 100],  # PCA components
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__max_depth': [3, 4, 5, 6, 7]
}

# Define a scorer for F1 score
macro_f1_scorer = make_scorer(f1_score, average='macro')

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, scoring=macro_f1_scorer, cv=3, verbose=2)

# Fit the grid search to the data
grid_search.fit(df_feature_train_scaled, df_target_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation macro F1 score: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(df_feature_test_scaled)
test_macro_f1 = f1_score(df_target_test, predictions, average='macro')
print(f"Test macro F1 score: {test_macro_f1:.2f}")

Fitting 3 folds for each of 80 candidates, totalling 240 fits
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=2000; total time=  20.8s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=2000; total time=  25.9s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=2000; total time=  25.8s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=1000; total time=   9.4s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=1000; total time=  10.5s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=1000; total time=  10.2s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=500; total time=   4.7s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=500; total time=   7.0s
[CV] END classifier__max_depth=3, classifier__n_estimators=50, pca__n_components=500; total 

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [20]:
# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(df_feature_test_scaled)
print(grid_search.best_params_)

{'classifier__max_depth': 5, 'classifier__n_estimators': 300, 'pca__n_components': 1000}


In [26]:
test_macro_f1 = f1_score(df_target_test, predictions, average='macro')
print(f"Test macro F1 score: {test_macro_f1:.2f}")

Test macro F1 score: 0.64


max_depth = 5
n_estimators = 300
pca, n_components = 1000

macro f1 score = 0.64

In [34]:
df_test = pd.read_csv('./data/test_tfidf_features.csv')
df_test.drop('id', axis=1, inplace=True)
df_test_scaled = scalar.transform(df_test)
# pca = PCA(n_components=1000)
# df_test_pca = pca.fit_transform(df_test_scaled)
pred = best_model.predict(df_test_scaled)
pred


array([0, 0, 1, ..., 1, 1, 0])

In [35]:
predictions_df = pd.DataFrame(pred, columns=['label'])
test_id = pd.read_csv('./data/test_tfidf_features.csv')['id']
submission_df = pd.concat([test_id, predictions_df], axis=1)
submission_df.to_csv('submission.csv', index=False)

In [36]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Define the functions
def prepare_feature(df_feature):
    if isinstance(df_feature, pd.DataFrame):
        np_feature = df_feature.to_numpy()
    else:
        np_feature = df_feature
    X = np.hstack((np.ones((np_feature.shape[0], 1)), np_feature))
    return X

def prepare_target(df_target):
    if isinstance(df_target, pd.DataFrame):
        np_target = df_target.to_numpy()
    else:
        np_target = df_target
    return np_target.ravel()

# Ensure split_data function is defined and returns correct splits
def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Assuming df_feature and df_target are already defined
# Prepare features and targets
prepared_features = prepare_feature(df_feature)
prepared_target = prepare_target(df_target)

# Split the data
df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(prepared_features, prepared_target, random_state=42, test_size=0.3)

# Standardize the data
scaler = StandardScaler()
df_feature_train_scaled = scaler.fit_transform(df_feature_train)
df_feature_test_scaled = scaler.transform(df_feature_test)

# Define the pipeline
pipeline = Pipeline([
    ('pca', PCA()), 
    ('classifier', xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.1, n_estimators=300))
])

# Create a parameter grid
param_grid = {
    'pca__n_components': [2000, 1000, 500, 100],  # Adjust based on your feature size
    'classifier__max_depth': [4, 5, 6]
}

# Define a scorer for F1 score
macro_f1_scorer = make_scorer(f1_score, average='macro')

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, scoring=macro_f1_scorer, cv=3, verbose=2)

# Fit the grid search to the data
grid_search.fit(df_feature_train_scaled, df_target_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation macro F1 score: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(df_feature_test_scaled)
test_macro_f1 = f1_score(df_target_test, predictions, average='macro')
print(f"Test macro F1 score: {test_macro_f1:.2f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ....classifier__max_depth=4, pca__n_components=2000; total time=  54.4s
[CV] END ....classifier__max_depth=4, pca__n_components=2000; total time=  50.4s
[CV] END ....classifier__max_depth=4, pca__n_components=2000; total time=  54.7s
[CV] END ....classifier__max_depth=4, pca__n_components=1000; total time=  34.3s
[CV] END ....classifier__max_depth=4, pca__n_components=1000; total time=  22.9s
[CV] END ....classifier__max_depth=4, pca__n_components=1000; total time=  22.6s
[CV] END .....classifier__max_depth=4, pca__n_components=500; total time=  12.7s
[CV] END .....classifier__max_depth=4, pca__n_components=500; total time=  12.6s
[CV] END .....classifier__max_depth=4, pca__n_components=500; total time=  12.9s
[CV] END .....classifier__max_depth=4, pca__n_components=100; total time=   3.2s
[CV] END .....classifier__max_depth=4, pca__n_components=100; total time=   2.4s
[CV] END .....classifier__max_depth=4, pca__n_co

In [52]:
df_test = pd.read_csv('./data/test_tfidf_features.csv')
df_test.drop('id', axis=1, inplace=True)
df_test_prepared = prepare_feature(df_test)

# Remove the bias term before scaling (if necessary)
# df_test_prepared_wo_bias = df_test_prepared[:, 1:]

# Use the same scaler and PCA fitted on the training data
df_test_scaled = scaler.transform(df_test_prepared)

In [53]:
pred = best_model.predict(df_test_scaled)
pred

array([0, 0, 1, ..., 1, 0, 0])

In [54]:
predictions_df = pd.DataFrame(pred, columns=['label'])
test_id = pd.read_csv('./data/test_tfidf_features.csv')['id']
submission_df = pd.concat([test_id, predictions_df], axis=1)
submission_df.to_csv('submission.csv', index=False)

---

In [2]:
!pip install imblearn

   ---------------------------------------- 0.0/258.3 kB ? eta -:--:--
   ---------------------------------------  256.0/258.3 kB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 258.3/258.3 kB 5.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer, classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Define the functions
def prepare_feature(df_feature):
    if isinstance(df_feature, pd.DataFrame):
        np_feature = df_feature.to_numpy()
    else:
        np_feature = df_feature
    return np_feature

def prepare_target(df_target):
    if isinstance(df_target, pd.DataFrame):
        np_target = df_target.to_numpy()
    else:
        np_target = df_target
    return np_target.ravel()

def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

# Assuming df_feature and df_target are already defined
# Prepare features and targets
prepared_features = prepare_feature(df_feature)
prepared_target = prepare_target(df_target)

# Split the data
df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(prepared_features, prepared_target, random_state=42, test_size=0.3)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
df_feature_train_res, df_target_train_res = smote.fit_resample(df_feature_train, df_target_train)

# Standardize the data
scaler = StandardScaler()
df_feature_train_scaled = scaler.fit_transform(df_feature_train_res)
df_feature_test_scaled = scaler.transform(df_feature_test)

# Define the pipeline
pipeline = Pipeline([
    ('pca', PCA()), 
    ('classifier', xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.1))  # Default values
])

# Create a parameter grid
param_grid = {
    'pca__n_components': [2000, 1000, 500, 100],  # Adjust based on your feature size
    'classifier__n_estimators': [100, 200, 300, 400],
    'classifier__max_depth': [4, 5, 6, 7, 8],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__subsample': [0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.8, 0.9, 1.0]
}

# Define a scorer for F1 score
macro_f1_scorer = make_scorer(f1_score, average='macro')

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, scoring=macro_f1_scorer, cv=3, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(df_feature_train_scaled, df_target_train_res)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation macro F1 score: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(df_feature_test_scaled)
test_macro_f1 = f1_score(df_target_test, predictions, average='macro')
print(f"Test macro F1 score: {test_macro_f1:.2f}")

# Print a detailed classification report
print(classification_report(df_target_test, predictions))

Fitting 3 folds for each of 2160 candidates, totalling 6480 fits


KeyboardInterrupt: 

## Support Vector Machine

In [60]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer, classification_report

# Define the functions
def prepare_feature(df_feature):
    if isinstance(df_feature, pd.DataFrame):
        np_feature = df_feature.to_numpy()
    else:
        np_feature = df_feature
    X = np.hstack((np.ones((np_feature.shape[0], 1)), np_feature))
    return X

def prepare_target(df_target):
    if isinstance(df_target, pd.DataFrame):
        np_target = df_target.to_numpy()
    else:
        np_target = df_target
    return np_target.ravel()

def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    indexes = df_feature.index.to_numpy()
    if random_state is not None:
        np.random.seed(random_state)
    test_index = np.random.choice(indexes, int(len(indexes) * test_size), replace=False)
    train_index = np.setdiff1d(indexes, test_index)
    
    df_feature_train = df_feature.loc[train_index, :]
    df_feature_test = df_feature.loc[test_index, :]
    df_target_train = df_target.loc[train_index, :]
    df_target_test = df_target.loc[test_index, :]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

# Assuming df_feature and df_target are already defined and preprocessed
# df_feature is in TF-IDF format with 5000 features
# df_target contains the target labels

# Prepare features and targets
prepared_features = prepare_feature(df_feature)
prepared_target = prepare_target(df_target)

# Split the data
df_feature_train, df_feature_test, df_target_train, df_target_test = split_data(df_feature, df_target, random_state=42, test_size=0.3)

# Standardize the data
scaler = StandardScaler(with_mean=False)  # with_mean=False because TF-IDF data is sparse
df_feature_train_scaled = scaler.fit_transform(df_feature_train)
df_feature_test_scaled = scaler.transform(df_feature_test)

# Define the pipeline
pipeline = Pipeline([
    ('pca', PCA()), 
    ('classifier', SVC(kernel='linear'))
])

# Create a parameter grid
param_grid = {
    'pca__n_components': [2000, 1000, 500, 100],  # Adjust based on your feature size
}

# Define a scorer for F1 score
macro_f1_scorer = make_scorer(f1_score, average='macro')

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, scoring=macro_f1_scorer, cv=5, verbose=2)

# Fit the grid search to the data
grid_search.fit(df_feature_train_scaled, df_target_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation macro F1 score: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(df_feature_test_scaled)
test_macro_f1 = f1_score(df_target_test, predictions, average='macro')
print(f"Test macro F1 score: {test_macro_f1:.2f}")

# Print a detailed classification report
print(classification_report(df_target_test, predictions))

Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\banno\OneDrive - Singapore University of Technology and Design\Desktop\machine_learning_1d\.venv\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
